<a href="https://colab.research.google.com/github/Vyom88/Toss-it/blob/main/WasteClassificationModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras,os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import cv2
from google.colab import drive
drive.mount('/content/drive')

from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
vgg_model.input

<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_1')>

In [ ]:
#Get Data
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(
    directory='/content/drive/MyDrive/Data/Train', target_size=(224, 224), class_mode="categorical") # CHANGE BATCH SIZE

tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory='/content/drive/MyDrive/Data/Test', target_size=(
    224, 224), class_mode="categorical")  # CHANGE BATCH SIZE

Found 24624 images belonging to 3 classes.
Found 6152 images belonging to 3 classes.


In [ ]:
# Freeze four convolution blocks
for layer in vgg_model.layers:
    layer.trainable = False
# Make sure you have frozen the correct layers
for i, layer in enumerate(vgg_model.layers):
    print(i, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False


In [ ]:
x = Flatten()(vgg_model.output)
prediction = Dense(3, activation='softmax')(x)
model = Model(inputs=vgg_model.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
#Add layers to train for our specific use-case
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(3, activation='softmax')


model = models.Sequential([
    vgg_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_1 (Dense)             (None, 50)                1254450   
                                                                 
 dense_2 (Dense)             (None, 20)                1020      
                                                                 
 dense_3 (Dense)             (None, 3)                 63        
                                                                 
Total params: 15,970,221
Trainable params: 1,255,533
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

checkpoint = ModelCheckpoint("/content/drive/MyDrive/Data/Model/model.h5", monitor='val_accuracy', verbose=1,
                             save_best_only=True, save_weights_only=False,
                             mode='auto',
                             #period=1
                             )

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=8,  restore_best_weights=True)

model.fit(x = traindata,
          validation_data = testdata,
          epochs=50,
          batch_size=32,
          steps_per_epoch=10,
          validation_steps=10,
          callbacks=[es, checkpoint])

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 9.8793 - accuracy: 0.6656  
Epoch 1: val_accuracy improved from -inf to 0.75313, saving model to /content/drive/MyDrive/Data/Model/model.h5
10/10 [==============================] - 337s 35s/step - loss: 9.8793 - accuracy: 0.6656 - val_loss: 8.7937 - val_accuracy: 0.7531
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 7.7307 - accuracy: 0.7719 
Epoch 2: val_accuracy improved from 0.75313 to 0.75937, saving model to /content/drive/MyDrive/Data/Model/model.h5
10/10 [==============================] - 302s 31s/step - loss: 7.7307 - accuracy: 0.7719 - val_loss: 9.2394 - val_accuracy: 0.7594
Epoch 3/50
10/10 [==============================] - ETA: 0s - loss: 8.2429 - accuracy: 0.7844 
Epoch 3: val_accuracy did not improve from 0.75937
10/10 [==============================] - 299s 31s/step - loss: 8.2429 - accuracy: 0.7844 - val_loss: 6.9546 - val_accuracy: 0.7594
Epoch 4/50
10/10 [=========================

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(x = traindata,
          validation_data = testdata,
          epochs=10,
          batch_size=32,
          callbacks=[es])

In [ ]:
#LOADING AND COMPILING TRAINED MODEL
trained_model = models.load_model('/content/drive/MyDrive/model.h5')
trained_model.compile(optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],)

In [ ]:
print(trained_model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
img_path = "/content/drive/MyDrive/Data/Test/R/R_14859.jpg"
classes = ['Recycling', "Organic", "Garbage"]
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.


    return img_tensor

new_image = load_image(img_path)


preds = trained_model.predict(new_image)

MaxPosition=np.argmax(preds)
prediction_label=classes[MaxPosition]

print(prediction_label)